# Capstone Project - The Battle of Neighborhoods
#### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

<!-- * [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion) -->

## Introduction: Business Problem <a name="introduction"></a>

__Is New York City more like Toronto?__

Specifically, this project will be targeted to people/business new to the city and interested in finding out the neighborhoods with specific venue or venues, and compare it with the other neighborhoods. It will also help analyse the businesses within city.

Primary objective of this project is to set up the process to compare Neighborhood across geographical location, and provide results that help us to analyze the selected Cities and its Neighborhoods compare to each other in terms of the likelihood of their neighborhoods and kind of venues they have. i.e. one can compare the cities in terms of the number of restaurants serving their favourite cuisine (# venues with in City), or neighborhoods which has venues of their choice (categories of venues with in neighborhood).

We will use our data science powers to clean,analyse and classify the data, and then prepare final report with few graphs and folium maps.

## Data <a name="data"></a>

We will consider all neighborhoods of Toronto, Canada and New York city of America for this project, to compare neighborhoods across both the cities and to analyze neighborhoods having similar kind of venues within same or different city. 

We will be using Foursquare API to find out top 10 venues for each neighborhood.  To explore and find out a nearby venues using foursquare, We will need geographical location (Address, Longitude, Latitude) data for each neighborhood, For this project we will use the data I have already compiled earlier while working on previous capstone project and lab assignments. 

We will then explore, segment and cluster neighborhoods based on characteristics of top 10 venues surrounding (i.e. within 500 meter radius) of neighborhood's geographical location.


----------------------------------------------- End of week 1 assignment -----------------------------------------------
<hr style="height:1px;border-width:0;color:white;background-color:brown">